In [1]:
import os
import sys
from pathlib import Path
sys.path.insert(1, os.path.realpath(os.path.pardir))

import torch
import torch.nn.functional as F
from torch import nn

import safetensors
from accelerate import notebook_launcher

import einops

from utils.data_utils import BrainDataset, get_tokenizer
from utils.train_utils import TrainConfig, run_train_model, count_parameters, simple_train_model

from transformers import GPT2Tokenizer

from models.bert import BrainBert, BertConfig
from models.vq_brain_per_channel import SoundStream

from dataclasses import dataclass
from simple_parsing.helpers import Serializable

from safetensors.torch import load_model


In [2]:
# device='cuda'
vq_config = dict(C=128, n_electrodes=1, levels=[8, 5, 5, 5])

vq_vae = SoundStream(**vq_config)
vq_vae = torch.compile(vq_vae)
vq_weights = "/drive/logs/kovalev/vq_brain/per_spikes_minmax_fsq_22M_ws_256_l1/step_3000_loss_0.0147.safetensors"
load_model(vq_vae, vq_weights)
vq_vae = vq_vae._orig_mod



self.codebook_size 1000


In [3]:
cfg = BertConfig(dim=128, window_size=768//4, n_electrodes=256, mask_ratio=0.75, n_layers=6)

model = BrainBert(cfg, vq_vae)

for param in model.tokenizer.parameters():
    param.requires_grad = False 

count_parameters(model)

BertConfig(window_size=192, n_electrodes=256, mask_ratio=0.75, tokenizer_downsample=4, n_layers=6, dim=128, hidden_dim=1024, head_dim=32, n_heads=16, n_kv_heads=16)
Encoder: number of parameters: 13.90M
Total: 13.90M, Trainable: 8.18M


(13901165, 8182120)

### Run training pipeline

In [ ]:
project_name = 'bert'

train_config = TrainConfig(exp_name='all_unpadded_loss_sep_vq_minmax_spikes',
                           mixed_precision=True,
                           batch_size=2, 
                           num_workers=3,
                           pin_memory=True, 
                           eval_interval=500, 
                           learning_rate=3e-4,
                           weight_decay=0, 
                           grad_clip=5,
                           lr_decay_iters=30_000, 
                           warmup_iters=1000
                          )
# peter path
# data_path = Path(r'C:\Users\peter\alvi\brain2text\competitionData')

# data_path = Path(r'D:\Work\brain-to-text-competition\data\competitionData')

data_path = Path("/drive/data/competitionData")
save_folder = Path("/drive/logs/kovalev")

tokenizer = GPT2Tokenizer.from_pretrained('gpt2')

train_dataset = BrainDataset(data_path / 'train', tokenize_function=get_tokenizer(tokenizer), max_input_len=768)
test_dataset = BrainDataset(data_path / 'test', tokenize_function=get_tokenizer(tokenizer), max_input_len=768)

# indices = torch.arange(1).repeat(64)
# train_dataset = torch.utils.data.Subset(test_dataset, indices)
# test_dataset = torch.utils.data.Subset(test_dataset, indices)

model = torch.compile(model)
args = (model, (train_dataset, test_dataset), train_config, project_name, save_folder)
notebook_launcher(run_train_model, args, num_processes=1)

Runed processing of the  /drive/data/competitionData/train


/opt/conda/envs/pytorch/lib/python3.10/site-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


len: 8800
max input len 768
Runed processing of the  /drive/data/competitionData/test
len: 880
max input len 768
Launching training on one GPU.


/opt/conda/envs/pytorch/lib/python3.10/site-packages/accelerate/accelerator.py:446: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['split_batches']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(split_batches=True)
  warnings.warn(
wandb: Currently logged in as: koval_alvi. Use `wandb login --relogin` to force relogin


Device for training:  cuda
Num devices:  1
Completed initialization of scheduler
********************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************************
overall_steps 500: 3.6049320697784424
val loss: 3.619947671890259
saved model:  step_500_loss_3.6199.safetensors
**********************

In [ ]:
save_folder = Path("/drive/logs/kovalev")

safetensors.torch.save_model(model, save_folder / 'model_mae_long_train.safetensors')

In [ ]:
model

In [ ]:
import torch._dynamo
torch._dynamo.config.suppress_errors = True

In [ ]:
model = model._orig_mod

In [ ]:
mae_weights = "/drive/logs/kovalev/fixed_abs_mae_11M_5M_spikes/step_34000_loss_0.0166.safetensors"

mae_model = SimpleMAE( SimpleEncoderConfig(), SimpleMAEConfig())
mae_model = torch.compile(mae_model)
safetensors.torch.load_model(mae_model, mae_weights)
mae_model = mae_model._orig_mod

brain_model = mae_model.encoder

In [ ]:
x = test_dataset[0][0]
device = 'cpu'
x = torch.from_numpy(x[None, :]).to(device)
print(x.shape)
loss, y, binary = model(x, masking_ratio=0.75,  return_preds=True)

x = x.detach().cpu()[0]
y = y.detach().cpu()[0]
binary = binary.detach().cpu()[0]

import matplotlib.pyplot as plt 

# plt.plot(y[1, :])
# plt.show()
# plt.plot(x[1, :])
# plt.show()
loss = F.l1_loss(y, x, reduction='none')
print(torch.mean(loss[loss>0]))

plt.show()
plt.imshow(binary[:64].T, aspect='auto')
# plt.colorbar()
plt.show()
plt.imshow(torch.clip(y[:64].T, 0, 1), aspect='auto')
plt.colorbar()
plt.show()
plt.imshow(x[:64].T, aspect='auto' )
plt.colorbar()


In [ ]:
plt.show()
plt.plot(binary[:32, 10])
plt.show()
plt.plot(y[:32, 10])
# plt.show()
plt.plot(x[:32, 10])